In [1]:
import os
os.environ["CUDA_VISIBLE-DEVICES"] = "0"

In [4]:
import numpy as np
import pickle
import gzip
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

# Requirements to Run This Notebook

### 1. Input Data
- **Training or Independent Data**: The data should be provided in `.dat` format. Length of peptides = 15aa
- **HLA Allele**: Aligned sequence form of HLA alleles, for example  file named -- "MHC2_prot_alignseq.dat"

### 2. Supporting Files
- ** C-alpha Matrix**: A pre-computed matrix which includs the  structural relationship between amino acids based on c-alpha distance

### 3. Output
- **Amino Acid Interaction Matrix**: 
  - For all used immunogenic peptides, the interaction matrix is generated.
  - This matrix serves as the initial output and is used for selecting the Optimal Number of Features (ONF).

### Including both HLA Chain and averaging their features

In [3]:
def matchDat(afflst, hladic, aadic):
    seqlst = []
    tablst = []
    header = []
    
    for affin in afflst:
        affstr = affin.strip().split('\t')
        
        # Check if the HLA type includes two chains (double alleles)
        if '/' in affstr[0]:
            hla_types = affstr[0].split('/')
        else:
            hla_types = [affstr[0]]
        
        feature_list = []
        for hla_type in hla_types:
            # Add the missing "HLA-" prefix if not present
            if not hla_type.startswith("HLA-"):
                hla_type = f"HLA-{hla_type}"
            
            if hla_type not in hladic:
                print(f"Missing HLA type: {hla_type}")
                continue
            
            hlaseq = hladic[hla_type]
            aaseq = affstr[1]
            #print(hladic.keys())
            tmp = []
            tmp0 = []
            for hlain in hlaseq:
                for aain in aaseq:
                    if hlain == 'X' or aain == 'X':
                        tmp0.append([0.0])
                    elif hlain == '*' or hlain == '.' or aain == 'X':
                        tmp0.append([0.0])
                    elif aain == 'U':
                        tmp0.append([aadic.get((hlain, 'C'), 0.0)])
                    elif aain == 'J':
                        aa1 = aadic.get((hlain, 'L'), 0.0)
                        aa2 = aadic.get((hlain, 'I'), 0.0)
                        aamax = max(aa1, aa2)
                        tmp0.append([aamax])
                    elif aain == 'Z':
                        aa1 = aadic.get((hlain, 'Q'), 0.0)
                        aa2 = aadic.get((hlain, 'E'), 0.0)
                        aamax = max(aa1, aa2)
                        tmp0.append([aamax])
                    elif aain == 'B':
                        aa1 = aadic.get((hlain, 'D'), 0.0)
                        aa2 = aadic.get((hlain, 'N'), 0.0)
                        aamax = max(aa1, aa2)
                        tmp0.append([aamax])
                    else:
                        tmp0.append([aadic.get((hlain, aain), 0.0)])
                tmp.append(tmp0)
                tmp0 = []
            
            feature_list.append(list(zip(*tmp)))
        
        # Combine the features if there are double alleles
        if len(feature_list) == 2:
            combined_features = np.mean(feature_list, axis=0)  #  Averaging both features
            seqlst.append(combined_features)
        elif len(feature_list) == 1:
            seqlst.append(feature_list[0])
        
        tablst.append(int(affstr[2]))
        header.append((affstr[0], affstr[1]))
    
    # Print shape of the sequence list array for debugging
    seqarray0 = np.array(seqlst, dtype=np.float32)
    print(f"seqarray0.shape: {seqarray0.shape}")  # Debugging line
    del seqlst
    
    if len(seqarray0.shape) < 3:
        raise ValueError("seqarray0 does not have the expected 3 dimensions.")
    
    a_seq2 = seqarray0.reshape(seqarray0.shape[0], seqarray0.shape[1] * seqarray0.shape[2])
    a_lab2 = np.array(tablst, dtype=np.float32)
    del tablst
    
    return (a_seq2, a_lab2), header


In [4]:
def HeaderOutput(lstin, outname):
    with open(outname, 'w') as outw:
        for lin in lstin:
            outw.write('\t'.join(map(str, lin)) + '\n')

def modifyMatrix(affydatin_test, seqdatin, outfile):
    hladicin = {x.strip().split('\t')[0]: list(x.strip().split('\t')[1]) for x in open(seqdatin).readlines()}
    aalst = open('../All_Data/Calpha.txt').readlines()
    aadicin = {}
    aaseq0 = aalst[0].strip().split('\t')
    for aain in aalst[1:]:
        aastr = aain.strip().split('\t')
        for i in range(1, len(aastr)):
            aadicin[(aaseq0[i-1], aastr[0])] = float(aastr[i])
    afflst = open(affydatin_test).readlines()
    d, test_header = matchDat(afflst, hladicin, aadicin)
    
    outname0 = affydatin_test
    outname2 = affydatin_test + '.header'
    
    with gzip.open(outfile, 'wb') as f:
        pickle.dump(d, f, protocol=2)
    
    HeaderOutput(test_header, outname2)

In [16]:
#print(hladic.keys())

In [17]:
## double allele MHC II included independent 200
modifyMatrix("../MHC_II/our_data_mhc_ii_independent_double_allle_200.dat",'../All_Data/MHC2_prot_alignseq.dat', 'Feature_MHC_ii_our_independent_200_double_allele_included')

Missing HLA type: HLA-DRA-0101
seqarray0.shape: (200, 15, 269, 1)


In [7]:
##double allele MHC II included training
modifyMatrix("../MHC_II/our_data_mhc_II_training_duplicated_checked_5960.dat",'../All_Data/MHC2_prot_alignseq.dat', 'Feature_MHC_ii_our_5960_duplicate_checked_double_allele_included')

Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
Missing HLA type: HLA-DRA-0101
seqarray0.shape: (5960, 15, 269, 1)


____________End________________

After ran above code, reading output compressed in zip file

In [8]:
with gzip.open('Feature_MHC_ii_our_independent_200_double_allele_included', 'rb') as f: ## Read the generated matrix, independent data
    array = pickle.load(f)
print(type(array[0]))

<class 'numpy.ndarray'>


In [9]:
array[0].shape # number of samples (200) and initial features 4035

(200, 4035)

In [10]:
array[0].shape

(200, 4035)

In [12]:
with gzip.open("Feature_MHC_ii_our_5960_duplicate_checked_double_allele_included") as f:
    array = pickle.load(f)


(5960,)

In [13]:
array[0].shape, array[1].shape

((5960, 4035), (5960,))